### 1. 视频读取接口

In [13]:
import cv2
import numpy as np
import time
import matplotlib.pyplot as plt
%matplotlib inline

def classification_with_vedio(model, image_size, video_name_or_link, decode):
    
    cap = cv2.VideoCapture(video_name_or_link)
    
    if(cap.isOpened() == False):
        print ("Failed to open " + video_name_or_link)
        return
    
    while True:
        # get fram
        ret, frame = cap.read()
        
        # check if the video is over
        if(ret != True):
            print ("Ending!")
            break
        
        # resize fram
        img_for_model = cv2.resize(frame, image_size)
        
        # change shape
        img_for_model= np.expand_dims(img_for_model,axis=0)
        
        # model predict
        start = time.time()
        preds = model.predict(img_for_model)
        end = time.time()
        # add text
        text = 'Predicted:' + decode(preds, top=1)[0][0][1] + ' time:' + str(end-start)
        
        # show frame
        cv2.putText(frame, text, (100, 200), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2, cv2.LINE_AA)
        cv2.imshow('image', frame)
        #cv2.moveWindow('image', 0, 0)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
        
    cap.release()
    cv2.destroyAllWindows()

### 2. 模型搭建接口

In [5]:
from keras.applications import resnet50,xception
from keras.layers import Input,Lambda

resNet_input_shape = (299,299,3)
res_x = Input(shape=resNet_input_shape)
res_x = Lambda(xception.preprocess_input)(res_x)
res_model = xception.Xception(include_top=True, weights='imagenet', input_tensor=res_x)

/home/riki/anaconda3/envs/py3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


###  3. 视频帧分类

In [14]:
classification_with_vedio(res_model, (299, 299), './animal.mp4', resnet50.decode_predictions)